CSD estimation is about twice as fast if you can use parameters from another file and avoid L-Curve. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings(action='once')

In [3]:
from sglxarray import load_trigger
from ecephys.xrsig import get_kcsd
import ecephys_analyses as ea

/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [4]:
import numpy as np
import seaborn as sns
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import json

In [5]:
def plot_epoched_profile(da, figsize=(36, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(da, xticklabels=da.time.values.round(), yticklabels=da.channel.values, cbar=False)
    ax.set(xticks=ax.get_xticks()[::2], yticks=ax.get_yticks()[::4], xlabel='Epoch center time (s)', ylabel='Channel')

In [6]:
def plot_profile(da, figsize=(36, 5), ylabel=None, negative_peaks=False):
    fig, ax = plt.subplots(figsize=figsize)
    channel_indices = np.arange(len(da.channel))
    sns.barplot(x=channel_indices, y=da, color='steelblue', ax=ax)
    ax.set(xlabel="Channel", ylabel=ylabel)
    ax.set_xticks(channel_indices);
    ax.set_xticklabels(da.channel.values, rotation=90);
    
    if negative_peaks:
        peaks, _ = find_peaks(-da.values, prominence=1000, distance=10)
    else:
        peaks, _ = find_peaks(da.values, prominence=1000, distance=10)
    sns.scatterplot(x=peaks, y=da.values[peaks], marker='x', color='red', ax=ax)

In [7]:
def write_file_report(csd, epoch_length, pdf_path):
    
    if csd.time.values.max() < epoch_length:
        return
    
    epoched_csd_nadirs = csd.coarsen(time=int(csd.fs *epoch_length), boundary='trim', coord_func={"time": "min"}).min()
    epoched_csd_variance = csd.coarsen(time=int(csd.fs *epoch_length), boundary='trim', coord_func={"time": "min"}).var()
    
    pdf_path = pdf_path.parent / (pdf_path.stem + f'-{epoch_length}s' + pdf_path.suffix)
    with PdfPages(pdf_path) as pdf:
        plot_epoched_profile(epoched_csd_variance)
        plt.title(f'CSD variance, {epoch_length}s epochs')
        pdf.savefig()
        plt.close()
        
        plot_epoched_profile(-epoched_csd_nadirs)
        plt.title(f'CSD nadirs, {epoch_length}s epochs')
        pdf.savefig()
        plt.close()

        for epoch in range(epoched_csd_nadirs.shape[1]):
            plot_profile(epoched_csd_variance[:, epoch], ylabel="CSD variance")
            plt.title(f'CSD variance, epoch: {epoch}, time: {epoched_csd_nadirs.time.values[epoch]}s')
            pdf.savefig()
            plt.close()
            
            plot_profile(epoched_csd_nadirs[:, epoch], ylabel="Deepest CSD sink (mA/mm)", negative_peaks=True)
            plt.title(f'CSD nadirs, epoch: {epoch}, time: {epoched_csd_nadirs.time.values[epoch]}s')
            pdf.savefig()
            plt.close()

        d = pdf.infodict()
        d['Title'] = f'{pdf_path.stem}'
        d['Author'] = 'Graham Findlay'
        d['Subject'] = 'CSD nadir profiles for tracking CA1 drift'
        d['CreationDate'] = datetime.now()

In [8]:
def write_alias_reports(subject, experiment, alias, probe, use_spw_params=True):
    drift_tracking_chans = ea.get_channels(subject, experiment, probe, "drift_tracking")

    if use_spw_params:
        spw_params_path = ea.get_experiment_subject_file("SPWRs", experiment, subject, "sharp_wave_detection_params.json")
        with open(spw_params_path) as spw_params_file:
            spw_params = json.load(spw_params_file)
            csd_params = spw_params["csd_params"]
    else:
        internal_reference = ea.get_channels(
            subject, experiment, probe, "internal_reference"
        )
    
    bin_paths = ea.get_lfp_bin_paths(subject, experiment, alias, probe=probe)
    pdf_paths = ea.get_project_counterparts("SPWRs", subject, bin_paths, ".csd.pdf")

    for bin_path, pdf_path in zip(bin_paths, pdf_paths):
        sig = load_trigger(bin_path, drift_tracking_chans)
        
        if use_spw_params:
            csd = get_kcsd(
                sig,
                np.arange(0, len(sig.channel)) * csd_params["electrode_pitch"],
                drop_chans=csd_params["channels_omitted_from_csd_estimation"],
                do_lcurve=False,
                gdx=csd_params["gdx"],
                R_init=csd_params["R"],
                lambd=csd_params["lambd"],
            ).swap_dims({"pos": "channel"})
        else:
            electrode_pitch = 0.020
            csd = get_kcsd(
                sig,
                np.arange(len(sig.channel)) * electrode_pitch,
                drop_chans=internal_reference,
                do_lcurve=True,
                gdx=electrode_pitch,
            ).swap_dims({"pos": "channel"})

        write_file_report(csd, 300, pdf_path)
        write_file_report(csd, 60, pdf_path)

        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [9]:
write_alias_reports("CNPIX12-Santiago", "conveyor_over_water", "full_48h", "imec1")

nChan: 385, nFileSamp: 18000039


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/kcsd/utility_functions.py:77: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.complex(0, n_src)]


12:51:09: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 18000038


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/kcsd/utility_functions.py:77: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.complex(0, n_src)]


12:59:52: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-3-2021/SpikeGLX/11-3-2021_g0/11-3-2021_g0_imec1/11-3-2021_g0_t1.imec1.lf.bin
nChan: 385, nFileSamp: 18000039


...

15:55:15: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-4-2021/SpikeGLX/11-4-2021_g0/11-4-2021_g0_imec1/11-4-2021_g0_t9.imec1.lf.bin
nChan: 385, nFileSamp: 18000038


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/kcsd/utility_functions.py:77: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.complex(0, n_src)]


16:03:06: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-4-2021/SpikeGLX/11-4-2021_g0/11-4-2021_g0_imec1/11-4-2021_g0_t10.imec1.lf.bin
nChan: 385, nFileSamp: 17997626


/home/gfindlay/miniconda3/envs/ecephys/lib/python3.10/site-packages/kcsd/utility_functions.py:77: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.complex(0, n_src)]


16:10:42: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX12-Santiago/11-4-2021/SpikeGLX/11-4-2021_g0/11-4-2021_g0_imec1/11-4-2021_g0_t11.imec1.lf.bin
